## 선형 회귀분석

### 관련 라이브러리 호출

In [ ]:
# 관련 라이브러리를 호출합니다.
import os
import joblib
import numpy as np
import pandas as pd
from scipy import stats
import pingouin as pg

In [ ]:
# 시각화 설정 모듈을 호출합니다.
from GraphicSetting import *

# EDA 시각화 및 통계 분석 관련 모듈을 호출합니다.
import hds_stats as hds

### 작업 경로 확인 및 변경

In [ ]:
# 현재 작업 경로를 확인합니다.
os.getcwd()

In [ ]:
# data 폴더로 작업 경로를 변경합니다.
os.chdir('../data')

In [ ]:
# 현재 작업 경로에 있는 폴더명과 파일명을 출력합니다.
sorted(os.listdir())

### 실습 데이터셋 준비

In [ ]:
# z 파일을 읽고 데이터프레임 df를 생성합니다.
df = joblib.load(filename = 'Used_Cars_Price_Prep.z')

In [ ]:
# df의 정보를 확인합니다.
df.info()

In [ ]:
# df의 처음 5행을 출력합니다.
df.head()

In [ ]:
# y절편 역할을 수행할 상수 1을 df의 두 번째 열로 삽입합니다.
df.insert(loc = 1, column = 'const', value = 1)

### 더미변수 생성

In [ ]:
# 범주형 입력변수로 더미변수를 생성합니다.
df = pd.get_dummies(data = df, 
                    prefix = [None], 
                    columns = ['FuelType'], 
                    drop_first = True, 
                    dtype = np.uint8)

In [ ]:
# df의 처음 10행을 출력합니다.
df.head(n = 10)

In [ ]:
# df의 열별 자료형을 확인합니다.
df.dtypes

In [ ]:
# 더미변수로 변환할 열이름으로 리스트를 생성합니다.
cols = ['MetColor']

In [ ]:
# 지정한 변수를 정수형으로 일괄 변환합니다.
df[cols] = df[cols].astype(np.uint8)

In [ ]:
# df의 열별 자료형을 확인합니다.
df.dtypes

### 실습 데이터셋 분할

In [ ]:
# 관련 라이브러리를 호출합니다.
from sklearn.model_selection import train_test_split

In [ ]:
# 전체 데이터의 70%를 훈련셋, 30%를 검증셋으로 분할합니다.
train, valid = train_test_split(df, train_size = 0.7, random_state = 0)

In [ ]:
# 목표변수명을 변수에 할당합니다.
yvar = 'Price'

### 입력변수와 목표변수 분리

In [ ]:
# 훈련셋을 입력변수 행렬과 목표변수 벡터로 분리합니다.
trainX = train.drop(columns = [yvar])
trReal = train[yvar].copy()

In [ ]:
# 훈련셋 목표변수의 평균을 확인합니다.
trReal.mean()

In [ ]:
# 검증셋을 입력변수 행렬과 목표변수 벡터로 분리합니다.
validX = valid.drop(columns = [yvar])
vaReal = valid[yvar].copy()

In [ ]:
# 검증셋 목표변수의 평균을 확인합니다.
vaReal.mean()

### 선형 회귀모형 적합 및 결과 확인

In [ ]:
# 훈련셋으로 선형 회귀모형을 적합합니다.
fit1 = hds.stat.ols(y = trReal, X = trainX)

In [ ]:
# fit1 모형의 적합 결과를 확인합니다.
fit1.summary()

### 회귀진단: 잔차 가정 확인

In [ ]:
# fit1 모형 잔차의 등분산성 검정을 실행합니다.
hds.stat.breushpagan(model = fit1)

### 회귀진단: 잔차 그래프

In [ ]:
# fit1 모형의 잔차 그래프를 그립니다.
hds.stat.regressionDiagnosis(model = fit1)

### 회귀진단: 잔차의 정규성 검정

In [ ]:
# 잔차의 커널 밀도 추정 곡선을 그립니다.
sns.kdeplot(x = fit1.resid, c = '0', lw = 1)
plt.axvline(x = 0, c = 'red', lw = 0.5);

In [ ]:
# 잔차의 정규성 검정을 실행합니다.
pg.normality(data = fit1.resid)

### 영향점 확인

In [ ]:
# 훈련셋의 관측값마다 영향점 정보를 갖는 데이터프레임을 생성합니다.
aug = hds.stat.augment(model = fit1)

In [ ]:
# aug의 처음 5행을 출력합니다.
aug.head()

In [ ]:
# aug를 cooksd 기준으로 내림차순 정렬하고 처음 5행을 출력합니다.
aug.sort_values(by = 'cooksd', ascending = False).head()

In [ ]:
# 쿡의 거리가 4/n(행 개수) 초과인 인덱스를 out_index에 할당합니다.
out_index = aug[aug['cooksd'].gt(4/trainX.shape[0])].index

### [참고] 영향점 시각화

In [ ]:
# Age와 Price의 산점도에 회귀직선을 그리고 빨간색 영향점을 추가합니다.
sns.regplot(x = trainX['Age'], y = trReal, ci = None, 
            scatter_kws = {'color': '0.8', 's': 10}, 
            line_kws = {'color': 'red'})
sns.scatterplot(data = df.loc[out_index], x = 'Age', y = 'Price', 
                c = 'red', s = 20, label = 'Outlier');

### 훈련셋에서 이상치 제거

In [ ]:
# out_index의 원소 개수를 확인합니다.
len(out_index)

In [ ]:
# trainX에서 인덱스가 out_index인 행을 삭제하고 행 개수를 확인합니다.
trainX = trainX.drop(index = out_index)
trainX.shape[0]

In [ ]:
# trReal에서 인덱스가 out_index인 행을 삭제하고 행 개수를 확인합니다.
trReal = trReal.drop(index = out_index)
trReal.shape[0]

### 선형 회귀모형 재적합 및 결과 확인

In [ ]:
# 이상치를 제거한 훈련셋으로 선형 회귀모형을 적합합니다.
fit2 = hds.stat.ols(y = trReal, X = trainX)

In [ ]:
# fit2 모형의 적합 결과를 확인합니다.
fit2.summary()

In [ ]:
# fit2 모형 잔차의 등분산성 검정을 실행합니다.
hds.stat.breushpagan(model = fit2)

In [ ]:
# fit2 모형의 잔차 그래프를 그립니다.
hds.stat.regressionDiagnosis(model = fit2)

### [참고] 더미변수의 시각적 이해

In [ ]:
# FuelType에 따라 회귀직선을 다르게 그립니다.
labels = ['Diesel', 'Petrol']
for i, v in enumerate(labels):
    sns.regplot(data = df[df['Petrol'].eq(i)], 
                x = 'Age', y = 'Price', 
                ci = None, label = v, 
                scatter_kws = {'s': 10, 'alpha': 0.2})
plt.legend(loc = 'best', title = 'FuelType');

### 목표변수의 추정값 생성

In [ ]:
# 훈련셋으로 fit1과 fit2 모형의 추정값을 생성하고 실제값과 비교합니다.
trPred1 = fit1.predict(exog = trainX)
trPred2 = fit2.predict(exog = trainX)
pd.DataFrame(data = {'Real': trReal, 'Pred1': trPred1, 'Pred2': trPred2})

In [ ]:
# 검증셋으로 fit1과 fit2 모형의 추정값을 생성하고 실제값과 비교합니다.
vaPred1 = fit1.predict(exog = validX)
vaPred2 = fit2.predict(exog = validX)
pd.DataFrame(data = {'Real': vaReal, 'Pred1': vaPred1, 'Pred2': vaPred2})

### 회귀모형 성능 평가

In [ ]:
# 훈련셋으로 fit1 모형의 성능지표를 출력합니다.
hds.stat.regmetrics(y_true = trReal, y_pred = trPred1)

In [ ]:
# 훈련셋으로 fit2 모형의 성능지표를 출력합니다.
hds.stat.regmetrics(y_true = trReal, y_pred = trPred2)

In [ ]:
# 검증셋으로 fit1 모형의 성능지표를 출력합니다.
hds.stat.regmetrics(y_true = vaReal, y_pred = vaPred1)

In [ ]:
# 검증셋으로 fit2 모형의 성능지표를 출력합니다.
hds.stat.regmetrics(y_true = vaReal, y_pred = vaPred2)

### [참고] MSE, MSLE, MAE, MAPE 관련 함수

In [ ]:
# 관련 라이브러리를 호출합니다.
from sklearn import metrics

In [ ]:
# 검증셋 추정값으로 MSE를 출력합니다.
metrics.mean_squared_error(y_true = vaReal, y_pred = vaPred2)

In [ ]:
# 검증셋 추정값으로 MSLE를 출력합니다.
metrics.mean_squared_log_error(y_true = vaReal, y_pred = vaPred2)

In [ ]:
# 검증셋 추정값으로 MAE를 출력합니다.
metrics.mean_absolute_error(y_true = vaReal, y_pred = vaPred2)

In [ ]:
# 검증셋 추정값으로 MAPE를 출력합니다.
metrics.mean_absolute_percentage_error(y_true = vaReal, y_pred = vaPred2)

### 다중공선성 확인

In [ ]:
# 분산팽창지수를 출력하고 다중공선성 입력변수를 확인합니다.
hds.stat.vif(X = trainX)

In [ ]:
# 다중공선성 입력변수가 있다고 가정하고 훈련셋에서 삭제합니다.
trainX1 = trainX.drop(columns = ['Petrol'])

In [ ]:
# 입력변수별 분산팽창지수를 다시 출력합니다.
hds.stat.vif(X = trainX1)

### 단계적방법으로 선형 회귀모형 적합

In [ ]:
# 단계적방법으로 선형 회귀모형을 적합합니다.
fit3 = hds.stat.stepwise(y = trReal, X = trainX, direction = 'both')

In [ ]:
# fit3 모형의 적합 결과를 확인합니다.
fit3.summary()

In [ ]:
# fit3 모형 잔차의 등분산성 검정을 실행합니다.
hds.stat.breushpagan(model = fit3)

In [ ]:
# fit3 모형의 잔차 그래프를 그립니다.
hds.stat.regressionDiagnosis(model = fit3)

### 표준화 회귀계수 확인

In [ ]:
# fit3 모형의 회귀계수를 출력합니다.
fit3.params

In [ ]:
# 표준화 회귀계수를 생성합니다.
beta_z = hds.stat.std_coefs(model = fit3)
beta_z

In [ ]:
# 표준화 회귀계수의 절대값을 오름차순 정렬한 결과를 출력합니다.
beta_z.abs().sort_values()

### 회귀모형 성능 평가

In [ ]:
# 검증셋으로 fit3 모형의 추정값을 생성합니다.
vaPred3 = fit3.predict(exog = validX)

In [ ]:
# 검증셋으로 fit3 모형의 성능지표를 출력합니다.
hds.stat.regmetrics(y_true = vaReal, y_pred = vaPred3)

In [ ]:
# 검증셋으로 fit2 모형의 성능지표와 비교합니다.
hds.stat.regmetrics(y_true = vaReal, y_pred = vaPred2)

## End of Document